In [1]:
from __future__ import division
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits import mplot3d
import matplotlib.cm as cm
import random
import math 
import mpmath as mp


def Poiseuille(Q):
    N1 = 80 #indeks 0 tablicy odpowiada -40
    N2 = 250
    Fi = np.zeros([N2,N1])
    Fi_N = np.zeros([N2,N1])
    W = np.zeros([N2,N1])#wirowosc
    W_N = np.zeros([N2,N1])
    dz = np.float64(0.01)
    y1 = -0.4
    y2 = 0.4
    mi = 1.0
    y = 0.0
    
    
    #war brzegowe    
    for i in range (0,N2):
        y = y2
        Fi[i][N1-1] = Q/2.0/mi*((y**3)/3.0+(y**2)/2.0*(y1+y2)+y1*y2*y)
        
    for j in range (1,N1-1):
        y = j*dz + y1
        Fi[0][j] = Q/2.0/mi*((y**3)/3.0+(y**2)/2.0*(y1+y2)+y1*y2*y)
        Fi[N2-1][j] = Q/2.0/mi*((y**3)/3.0+(y**2)/2.0*(y1+y2)+y1*y2*y)
        
        
    for i in range (95,106):
        y = y1
        Fi[i][50] = Q/2.0/mi*((y**3)/3.0+(y**2)/2.0*(y1+y2)+y1*y2*y)
    
    for i in range (0,95):
        y = y1
        Fi[i][0] = Q/2.0/mi*((y**3)/3.0+(y**2)/2.0*(y1+y2)+y1*y2*y)
    
    for i in range (106,N2):
        y = y1
        Fi[i][0] = Q/2.0/mi*((y**3)/3.0+(y**2)/2.0*(y1+y2)+y1*y2*y)
        
    
    for j in range (0,51):
        y = y1
        Fi[95][j] = Q/2.0/mi*((y**3)/3.0+(y**2)/2.0*(y1+y2)+y1*y2*y)
        Fi[105][j] = Q/2.0/mi*((y**3)/3.0+(y**2)/2.0*(y1+y2)+y1*y2*y)
    
        
    for i in range (0,N2):
        W[i][N1-1] = 2*(Fi[i][N1-2]-Fi[i][N1-1])/dz**2
        W[i][0] = 2*(Fi[i][1]-Fi[i][0])/dz**2

    
        
    Fi_N = Fi
    W_N = W
    #50
    
    for p in range (0,10000):
        
        for i in range (0,N2):
            W[i][N1-1] = 2.0*(Fi[i][N1-2]-Fi[i][N1-1])/dz**2
                       

        for j in range (0,50):
            W[95][j] = 2.0*(Fi[94][j]-Fi[95][j])/dz**2
            W[105][j] = 2.0*(Fi[106][j]-Fi[105][j])/dz**2
        
        for i in range (96,105):
            W[i][50] = 2.0*(Fi[i][51]-Fi[i][50])/dz**2
    
        for i in range (0,95):
            W[i][0] = 2.0*(Fi[i][1]-Fi[i][0])/dz**2

        for i in range (106,N2):
            y = y1
            W[i][0] = 2.0*(Fi[i][1]-Fi[i][0])/dz**2
        
        W[95][50] = 0.5*(W[94][50]+W[95][51])
        W[105][50] = 0.5*(W[106][50]+W[105][51])
        
        Fi_N = Fi
        W_N = W
        for i in range (1,N2-1):
            for j in range (1,N1-1):
                if (i>=95 and i<=105 and j<=50):
                    Fi_N[i][j] = Fi[i][j]
                else:
                    pom = np.float64(0.0)
                    Fi_N[i][j] = (Fi[i+1][j]+Fi[i-1][j]+Fi[i][j-1]+Fi[i][j+1] - W[i][j]*(dz**2))/4.0
                    pom = (W[i+1][j]+W[i-1][j]+W[i][j-1]+W[i][j+1])/4.0
                    pom = pom - ((Fi[i][j+1]-Fi[i][j-1])*(W[i+1][j]-W[i-1][j])-(Fi[i+1][j]-Fi[i-1][j])*(W[i][j+1]-W[i][j-1]))/16.0
                    W_N[i][j] = pom
        Fi=Fi_N
        W = W_N
        

    print("END")
    return Fi

In [2]:
def velocity_map(Fi,Q):
    N1 = 80
    N2 = 250
    U = np.zeros([N2,N1])
    V = np.zeros([N2,N1])
    for i in range (0,N2-1):
        for j in range (0,N1-1):
            if (i>=95 and i<=104 and j<50):
                U[i][j] = 0.0   #pionowy
                V[i][j] = 0.0
            else:
                U[i][j] = Fi[i][j+1] - Fi[i][j]   #pionowy
                V[i][j] = -(Fi[i+1][j] - Fi[i][j])
    
    for j in range (0,N1):
        U[N2-1][j] = U[N2-2][j]
        V[N2-1][j] = V[N2-2][j]
    
    for i in range (0,N2):
        U[i][N1-1]=U[i][N1-2]
        V[i][N1-1]=V[i][N1-2]
        
    for j in range(0,50):
        U[95][j] = U[94][j]
        V[95][j] = V[94][j]
    
    X,Y,Y2 = [],[],[]

    for i in range (95,105):
        U[i][50] = U[i][51]
        V[i][50] = V[i][51]
        X.append((i+1)*0.01-1.0)
        Y.append(0.1)
        Y2.append(-1.0)
    
    plt.figure(figsize=(20.0,10.0))
    plt.title('U(x,y) [pionowa] Q = {}'.format(Q))
    plt.xlabel('x')
    plt.ylabel('y')
    #plt.axis([0,1,0,5])
    plt.fill_between(X,Y,Y2, color = 'black')
    plt.imshow(np.rot90(V), extent=(-1.0, 1.5, -0.4, 0.4), cmap=cm.seismic, interpolation='nearest', aspect='auto', vmin = -np.max(V),vmax=np.max(V))
    plt.colorbar()
    plt.show()

    plt.figure(figsize=(20.0,10.0))
    plt.title('V(x,y) [pozioma]Q = {}'.format(Q))
    plt.xlabel('x')
    plt.ylabel('y')
    #plt.axis([0,1,0,5])
    plt.fill_between(X,Y,Y2, color = 'black')
    plt.imshow(np.rot90(U), extent=(-1.0, 1.5, -0.4, 0.4), cmap=cm.seismic, interpolation='nearest', aspect='auto', vmin = -np.max(U),vmax=np.max(U))
    plt.colorbar()
    plt.show()
    
    return 0
    

In [3]:
def velocity_map_lines(Fi,Q):
    plt.figure(figsize=(20.0,10.0))
    plt.title('$\psi$ (Q = {})' .format(Q) )

    X,Y,Y2 = [],[],[]

    plt.xlabel('x')
    plt.ylabel('y')
    for i in range (95,105):
        X.append(i*0.01-1.0)
        Y.append(0.1)
        Y2.append(-1.0)
    plt.xlim(-1.0,1.5)
    plt.ylim(-0.4,.4)
    plt.fill_between(X,Y,Y2, color = 'black')

    #plt.axis([0,1,0,5])
    plt.contour(Fi.T,30,extent=(-1.0,1.5, -0.4, 0.4))
    plt.colorbar()
    plt.show()

In [ ]:
N1 = 80 #indeks 0 tablicy odpowiada -40
N2 = 250
Fi0 = np.zeros([N2,N1])
Fi1 = np.zeros([N2,N1])
Fi2 = np.zeros([N2,N1])
Fi3 = np.zeros([N2,N1])
Fi4 = np.zeros([N2,N1])

Fi0 = Poiseuille(-1.0)
Fi1 = Poiseuille(-10.0)
Fi2 = Poiseuille(-100.0)
Fi3= Poiseuille(-200.0)
Fi4 = Poiseuille(-400.0)




END
END
END
END


In [ ]:
velocity_map_lines(Fi0,-1.0)

In [ ]:
velocity_map_lines(Fi1,-10.0)

In [ ]:
velocity_map_lines(Fi2,-100.0)

In [ ]:
velocity_map_lines(Fi3,-200.0)

In [ ]:
velocity_map_lines(Fi4,-400.0)

In [ ]:
velocity_map(Fi0,-1.0)

In [ ]:
velocity_map(Fi1,-10.0)

In [ ]:
velocity_map(Fi2,-100.0)

In [ ]:
velocity_map(Fi3,-200.0)

In [ ]:
velocity_map(Fi4,-400.0)